# [Module 4.1] 엣지 디바이스(EC2) 추가

이 노트북은 아래와 같은 주요 작업을 합니다.
새로운 엣지 디바이스가 생성될시에 아래와 같은 작업이 필요 하다는 것을 의미 합니다.

#### 1.EC2 인스턴스 생성
#### 2.Device Fleet에 엣지 디바이스 등록
#### 3.sagemaker edge manager agent를 EC2 인스턴스에 다운로드
- Agent 소프트웨서 다운로드, 압출 해제 및 바이너리 파일의 퍼미션 조정 함

#### 4.릴리스 버킷에서 모델 서명 루트 인증서를 다운로드
- IoT Certificate 다운로드
    - AmazonRootCA1.pem, iot.pem.crt, iot_key.pem.key s3에 업로드 
    - 위 파일을 EC2에 다운로드

#### 5.모델 패키징 파일 다운로드 및 추론할 샘플 이미지 다운로드

#### 6.sagemaker edge agent configuration file 생성, S3 업로드 및 EC2에 다운로드

#### 7.Launch Edge Agent 실행

#### 8.모델 로딩

#### 9.모델 추론



In [1]:
import sagemaker
from sagemaker import get_execution_role
import boto3
import botocore
import json

role = get_execution_role()
sess = sagemaker.Session()
region = boto3.Session().region_name

이전 노트북에서 사용한 변수를 로딩 합니다.

In [2]:
%store -r

In [3]:
folder = 'DEMO-Sandbox-Sagemaker-Edge'
compilation_output_sub_folder = folder + '/compilation-output'
iot_folder = folder + '/iot'

# S3 Location to save the model artifact after compilation
# s3_compilation_output_location = 's3://{}/{}'.format(bucket, compilation_output_sub_folder)

### EC2 Instance 생성
여기서는 추가적인 EC2 인스턴스를 생성 합니다.
- 이전 노트북에서 생성한 key를 재 사용
- 이전 노트북에서 생성한 ec2_profile_name를 재 사용

In [4]:
ec2_client = boto3.client('ec2', region_name=region)
print("ec2_profile_name: ", ec2_profile_name)

ec2_profile_name:  EdgeManager-Demo-EC2-gonsoo


In [5]:
ami_map = {
    'us-east-1': 'ami-063585f0e06d22308',
    'us-east-2': 'ami-01bd6a1621a6968d7',
    'us-west-2': 'ami-0bc87a16c757a7f07',
    'eu-central-1': 'ami-01227276a4e5a4a31',
    'ap-northeast-1': 'ami-03b8cfea5460e4881',
    'eu-west-1': 'ami-006ff58f5247c50eb'
}

In [6]:
ec2_instance = ec2_client.run_instances(
     ImageId=ami_map[region],
     MinCount=1,
     MaxCount=1,
     InstanceType='c5.large',
     KeyName= key_name,
     IamInstanceProfile={
        'Name': ec2_profile_name}
)

In [7]:
instance_id = ec2_instance['Instances'][0]['InstanceId'] # will used for running inference later
print("EC2 Instance ID: ", instance_id)

EC2 Instance ID:  i-00ed8289303d74580


### device fleet 에 엣지 디바이스 등록

In [8]:
sagemaker_client = boto3.client('sagemaker', region_name=region)

In [9]:
print("device_fleet_name: ", device_fleet_name)
print("iot_thing_name: ", iot_thing_name)

device_fleet_name:  demo2device2fleet1616397500
iot_thing_name:  sagemaker-edge-thing-demo-sandbox


In [10]:
import time
device_name = "sagemaker-edge-demo-device" + str(time.time()).split('.')[0] # device name should be 36 charactors

sagemaker_client.register_devices(
    DeviceFleetName=device_fleet_name,
    Devices=[
        {          
            "DeviceName": device_name,
            "IotThingName": iot_thing_name,
            "Description": "this is a sample virtual device"
        }
    ]
)

{'ResponseMetadata': {'RequestId': 'b54b55af-029f-4ba8-aa2e-e2cbede04a6a',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'b54b55af-029f-4ba8-aa2e-e2cbede04a6a',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Mon, 22 Mar 2021 07:33:20 GMT'},
  'RetryAttempts': 0}}

## Edge 디바이스 세팅

In [11]:
ssm_client = boto3.client('ssm', region_name=region)

### 세이지메이커 에지 관리자 에이전트 바이너리 examples를 EC2 인스턴스로 다운로드합니다.

sagemaker 에지 릴리스 버킷에서 바이너리의 최신 버전을 가져 오십시오.[추론 엔진 (에지 관리자 에이전트)](https://docs.aws.amazon.com/sagemaker/latest/dg/edge-device-fleet-about.html) 에 대한 자세한 내용을 참조하십시오.

조금 더 자세한 내용은 이전 노트북을 참고 하세요.

In [12]:
release_bucket_map = {
    'armv8' : 'sagemaker-edge-release-store-us-west-2-linux-armv8',
    'linux' : 'sagemaker-edge-release-store-us-west-2-linux-x64',
    'win64' : 'sagemaker-edge-release-store-us-west-2-windows-x64',
    'win32' : 'sagemaker-edge-release-store-us-west-2-windows-x86',
}

In [13]:
# In this example, we will run inference on linux platform
release_bucket = release_bucket_map['linux']

In [14]:
!aws s3 ls s3://$release_bucket/Releases/ | sort -r

                           PRE 1.20210305.a4bc999/
                           PRE 1.20201218.81f481f/
                           PRE 1.20201207.02d0e97/
2020-12-02 07:31:22          0 


In [15]:
version = "1.20210305.a4bc999"

In [18]:
response = ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName="AWS-RunShellScript",
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Parameters={
        'commands':[
            "#!/bin/bash",
            "mkdir /demo",
            "aws s3 cp s3://{}/Releases/{}/{}.tgz demo.tgz".format(release_bucket, version, version),
            "tar -xf demo.tgz -C /demo",
            "cd /demo/bin",
            "chmod +x sagemaker_edge_agent_binary",
            "chmod +x sagemaker_edge_agent_client_example"
        ]
    }
)

위의 EC2 인스턴스가 생성될때가지 약 30초를 대기 합니다. EC2 컨솔에 가서 실행 상태인지 확인해보세요.

In [21]:
import time
time.sleep(30)

In [19]:
ssm_client.get_command_invocation(
    CommandId=response['Command']['CommandId'],
    InstanceId=instance_id,
)

{'CommandId': '5bdbf388-1469-4d36-9fcc-0e70b6c707e8',
 'InstanceId': 'i-00ed8289303d74580',
 'Comment': '',
 'DocumentName': 'AWS-RunShellScript',
 'DocumentVersion': '',
 'PluginName': 'aws:runShellScript',
 'ResponseCode': 0,
 'ExecutionStartDateTime': '2021-03-22T07:34:45.644Z',
 'ExecutionElapsedTime': 'PT6.516S',
 'ExecutionEndDateTime': '2021-03-22T07:34:51.644Z',
 'Status': 'Success',
 'StatusDetails': 'Success',
 'StandardOutputContent': 'Completed 256.0 KiB/9.1 MiB (334.2 KiB/s) with 1 file(s) remaining\rCompleted 512.0 KiB/9.1 MiB (628.8 KiB/s) with 1 file(s) remaining\rCompleted 768.0 KiB/9.1 MiB (922.3 KiB/s) with 1 file(s) remaining\rCompleted 1.0 MiB/9.1 MiB (1.2 MiB/s) with 1 file(s) remaining    \rCompleted 1.2 MiB/9.1 MiB (1.4 MiB/s) with 1 file(s) remaining    \rCompleted 1.5 MiB/9.1 MiB (1.7 MiB/s) with 1 file(s) remaining    \rCompleted 1.8 MiB/9.1 MiB (2.0 MiB/s) with 1 file(s) remaining    \rCompleted 1.9 MiB/9.1 MiB (2.1 MiB/s) with 1 file(s) remaining    \rCompl

## 릴리스 버킷에서 모델 서명 루트 인증서 및 모델을 EC2로 다운로드

In [22]:
response = ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName="AWS-RunShellScript",
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Parameters={
        'commands':[
            "#!/bin/bash",
            "cd /demo",
            "mkdir certificates",
            "aws s3 cp s3://{}/Certificates/{}/{}.pem certificates".format(release_bucket, region, region)
        ]
    }
)

In [24]:
ssm_client.get_command_invocation(
    CommandId=response['Command']['CommandId'],
    InstanceId=instance_id,
)

{'CommandId': '151fbb29-49dc-451d-985b-c4ad515dd67c',
 'InstanceId': 'i-00ed8289303d74580',
 'Comment': '',
 'DocumentName': 'AWS-RunShellScript',
 'DocumentVersion': '',
 'PluginName': 'aws:runShellScript',
 'ResponseCode': 0,
 'ExecutionStartDateTime': '2021-03-22T07:37:07.793Z',
 'ExecutionElapsedTime': 'PT2.07S',
 'ExecutionEndDateTime': '2021-03-22T07:37:09.793Z',
 'Status': 'Success',
 'StatusDetails': 'Success',
 'StandardOutputContent': 'Completed 1.3 KiB/1.3 KiB (2.6 KiB/s) with 1 file(s) remaining\rdownload: s3://sagemaker-edge-release-store-us-west-2-linux-x64/Certificates/us-east-2/us-east-2.pem to certificates/us-east-2.pem\n',
 'StandardOutputUrl': 'https://s3.us-east-2.amazonaws.com/sagemaker-us-east-2-638683464862/DEMO-Sandbox-Sagemaker-Edge/151fbb29-49dc-451d-985b-c4ad515dd67c/i-00ed8289303d74580/awsrunShellScript/0.awsrunShellScript/stdout',
 'StandardErrorContent': 'mkdir: cannot create directory ‘certificates’: File exists\n',
 'StandardErrorUrl': 'https://s3.us-eas

IoT certificates, private key 및 모델 패키징 아티펙트 다운로드

In [25]:
print(root_ca_path)
print(device_cert_path)
print(device_key_path)
print(keras_img_path)
print(keras_model_data)
print(keras_model_package)

s3://sagemaker-us-east-2-638683464862/DEMO-Sandbox-Sagemaker-Edge/iot/AmazonRootCA1.pem
s3://sagemaker-us-east-2-638683464862/DEMO-Sandbox-Sagemaker-Edge/iot/iot.pem.crt
s3://sagemaker-us-east-2-638683464862/DEMO-Sandbox-Sagemaker-Edge/iot/iot_key.pem.key
s3://sagemaker-us-east-2-638683464862/DEMO-Sandbox-Sagemaker-Edge/iot/keras.bmp
s3://sagemaker-us-east-2-638683464862/DEMO-Sandbox-Sagemaker-Edge/compilation-output/keras-model-1.0.tar.gz
keras-model-1.0.tar.gz


In [27]:
response = ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName="AWS-RunShellScript",
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Parameters={
        'commands':[
            "#!/bin/bash",
            "cd /demo",
            "mkdir iot-credentials",
            "cd iot-credentials",
            "aws s3 cp " + root_ca_path + " .",  
            "aws s3 cp " + device_cert_path + " .",
            "aws s3 cp " + device_key_path + " .",
            "cd /demo",
            "aws s3 cp " + keras_img_path + " .",
            "aws s3 cp " + keras_model_data + " .",
            "mkdir keras_model",
            "tar -xf " + keras_model_package + " -C keras_model"
        ]
    }
)

In [29]:
ssm_client.get_command_invocation(
    CommandId=response['Command']['CommandId'],
    InstanceId=instance_id,
)

{'CommandId': 'cf541fde-c8fb-4e0e-a252-6a9c98e9f875',
 'InstanceId': 'i-00ed8289303d74580',
 'Comment': '',
 'DocumentName': 'AWS-RunShellScript',
 'DocumentVersion': '',
 'PluginName': 'aws:runShellScript',
 'ResponseCode': 0,
 'ExecutionStartDateTime': '2021-03-22T07:38:41.906Z',
 'ExecutionElapsedTime': 'PT4.256S',
 'ExecutionEndDateTime': '2021-03-22T07:38:45.906Z',
 'Status': 'Success',
 'StatusDetails': 'Success',
 'StandardOutputContent': 'Completed 1.2 KiB/1.2 KiB (17.4 KiB/s) with 1 file(s) remaining\rdownload: s3://sagemaker-us-east-2-638683464862/DEMO-Sandbox-Sagemaker-Edge/iot/AmazonRootCA1.pem to ./AmazonRootCA1.pem\nCompleted 1.2 KiB/1.2 KiB (20.5 KiB/s) with 1 file(s) remaining\rdownload: s3://sagemaker-us-east-2-638683464862/DEMO-Sandbox-Sagemaker-Edge/iot/iot.pem.crt to ./iot.pem.crt\nCompleted 1.6 KiB/1.6 KiB (27.4 KiB/s) with 1 file(s) remaining\rdownload: s3://sagemaker-us-east-2-638683464862/DEMO-Sandbox-Sagemaker-Edge/iot/iot_key.pem.key to ./iot_key.pem.key\nComp

## sagemaker edge manager agent 설정

sagemaker edge agent configuration file 생성

In [30]:
print("device_name: ", device_name)
print("device_fleet_name: ", device_fleet_name)
print("region: ", region)
print("endpoint: ", endpoint)
print("bucket: ", bucket)

device_name:  sagemaker-edge-demo-device1616398400
device_fleet_name:  demo2device2fleet1616397500
region:  us-east-2
endpoint:  https://c1er7obc0yilbm.credentials.iot.us-east-2.amazonaws.com/role-aliases/SageMakerEdge-demo2device2fleet1616397500/credentials
bucket:  sagemaker-us-east-2-638683464862


In [31]:
sagemaker_edge_config = {
    "sagemaker_edge_core_device_uuid": device_name,
    "sagemaker_edge_core_device_fleet_name": device_fleet_name,
    "sagemaker_edge_core_capture_data_buffer_size": 30,
    "sagemaker_edge_core_capture_data_batch_size": 10,
    "sagemaker_edge_core_capture_data_push_period_seconds": 4,
    "sagemaker_edge_core_folder_prefix": "demo_capture",
    "sagemaker_edge_core_region": region,
    "sagemaker_edge_core_root_certs_path": "/demo/certificates",
    "sagemaker_edge_provider_aws_ca_cert_file": "/demo/iot-credentials/AmazonRootCA1.pem",
    "sagemaker_edge_provider_aws_cert_file": "/demo/iot-credentials/iot.pem.crt",
    "sagemaker_edge_provider_aws_cert_pk_file": "/demo/iot-credentials/iot_key.pem.key",
    "sagemaker_edge_provider_aws_iot_cred_endpoint": endpoint,
    "sagemaker_edge_provider_provider": "Aws",
    "sagemaker_edge_provider_s3_bucket_name": bucket,
    "sagemaker_edge_core_capture_data_destination": "Cloud"
}

In [32]:
edge_config_file = open("sagemaker_edge_config.json", "w")
json.dump(sagemaker_edge_config, edge_config_file, indent = 6)
edge_config_file.close()

In [33]:
! head -n100 sagemaker_edge_config.json

{
      "sagemaker_edge_core_device_uuid": "sagemaker-edge-demo-device1616398400",
      "sagemaker_edge_core_device_fleet_name": "demo2device2fleet1616397500",
      "sagemaker_edge_core_capture_data_buffer_size": 30,
      "sagemaker_edge_core_capture_data_batch_size": 10,
      "sagemaker_edge_core_capture_data_push_period_seconds": 4,
      "sagemaker_edge_core_folder_prefix": "demo_capture",
      "sagemaker_edge_core_region": "us-east-2",
      "sagemaker_edge_core_root_certs_path": "/demo/certificates",
      "sagemaker_edge_provider_aws_ca_cert_file": "/demo/iot-credentials/AmazonRootCA1.pem",
      "sagemaker_edge_provider_aws_cert_file": "/demo/iot-credentials/iot.pem.crt",
      "sagemaker_edge_provider_aws_cert_pk_file": "/demo/iot-credentials/iot_key.pem.key",
      "sagemaker_edge_provider_aws_iot_cred_endpoint": "https://c1er7obc0yilbm.credentials.iot.us-east-2.amazonaws.com/role-aliases/SageMakerEdge-demo2device2fleet1616397500/credentials",
      "sagemaker_edge_provid

In [34]:
config_path = sess.upload_data('sagemaker_edge_config.json', bucket, iot_folder)
print(config_path)

s3://sagemaker-us-east-2-638683464862/DEMO-Sandbox-Sagemaker-Edge/iot/sagemaker_edge_config.json


In [35]:
response = ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName="AWS-RunShellScript",
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Parameters={
        'commands':[
            "#!/bin/bash",
            "aws s3 cp " + config_path + ' /demo'
        ]
    }
)

In [37]:
ssm_client.get_command_invocation(
    CommandId=response['Command']['CommandId'],
    InstanceId=instance_id,
)

{'CommandId': '5b1e0886-5d5b-4fe1-b480-211e80313f99',
 'InstanceId': 'i-00ed8289303d74580',
 'Comment': '',
 'DocumentName': 'AWS-RunShellScript',
 'DocumentVersion': '',
 'PluginName': 'aws:runShellScript',
 'ResponseCode': 0,
 'ExecutionStartDateTime': '2021-03-22T07:39:10.395Z',
 'ExecutionElapsedTime': 'PT0.9S',
 'ExecutionEndDateTime': '2021-03-22T07:39:10.395Z',
 'Status': 'Success',
 'StatusDetails': 'Success',
 'StandardOutputContent': 'Completed 1.1 KiB/1.1 KiB (15.0 KiB/s) with 1 file(s) remaining\rdownload: s3://sagemaker-us-east-2-638683464862/DEMO-Sandbox-Sagemaker-Edge/iot/sagemaker_edge_config.json to ../../../../demo/sagemaker_edge_config.json\n',
 'StandardOutputUrl': 'https://s3.us-east-2.amazonaws.com/sagemaker-us-east-2-638683464862/DEMO-Sandbox-Sagemaker-Edge/5b1e0886-5d5b-4fe1-b480-211e80313f99/i-00ed8289303d74580/awsrunShellScript/0.awsrunShellScript/stdout',
 'StandardErrorContent': '',
 'StandardErrorUrl': 'https://s3.us-east-2.amazonaws.com/sagemaker-us-east-2

### 모델 서명 루트 인증서의 권한 변경
/demo/certificates/us-east-2.pem 를 Read Only 로 변경해야 합니다. 아래 작업을 안할 시에는 Agent 를 런칭시에 에러가 발생합니다.
<br>

#### **[중요] 현재 리젼이 us-east-2 가 아니면 아래 코드에서 <region>.pem을 변경해야 합니다.**

In [40]:
cert_permission_out = ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName="AWS-RunShellScript",
    TimeoutSeconds=24*60*60,
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Parameters={
        'commands':[
            "cd /demo/certificates",
            "sudo chmod 444 us-east-2.pem", # 리젼이 다르면 변경 필요
        ]
    }
)

In [42]:
ssm_client.get_command_invocation(
    CommandId=cert_permission_out['Command']['CommandId'],
    InstanceId=instance_id,
)

{'CommandId': 'fae273ec-e96f-4b1f-81ca-5e115ea9c173',
 'InstanceId': 'i-00ed8289303d74580',
 'Comment': '',
 'DocumentName': 'AWS-RunShellScript',
 'DocumentVersion': '',
 'PluginName': 'aws:runShellScript',
 'ResponseCode': 0,
 'ExecutionStartDateTime': '2021-03-22T07:42:29.923Z',
 'ExecutionElapsedTime': 'PT0.199S',
 'ExecutionEndDateTime': '2021-03-22T07:42:29.923Z',
 'Status': 'Success',
 'StatusDetails': 'Success',
 'StandardOutputContent': '',
 'StandardOutputUrl': 'https://s3.us-east-2.amazonaws.com/sagemaker-us-east-2-638683464862/DEMO-Sandbox-Sagemaker-Edge/fae273ec-e96f-4b1f-81ca-5e115ea9c173/i-00ed8289303d74580/awsrunShellScript/0.awsrunShellScript/stdout',
 'StandardErrorContent': '',
 'StandardErrorUrl': 'https://s3.us-east-2.amazonaws.com/sagemaker-us-east-2-638683464862/DEMO-Sandbox-Sagemaker-Edge/fae273ec-e96f-4b1f-81ca-5e115ea9c173/i-00ed8289303d74580/awsrunShellScript/0.awsrunShellScript/stderr',
 'CloudWatchOutputConfig': {'CloudWatchLogGroupName': '',
  'CloudWatchO

## Sagemaker Edge Agent 런칭

#### SSM 으로 실행

In [43]:
agent_out = ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName="AWS-RunShellScript",
    TimeoutSeconds=24*60*60,
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Parameters={
        'commands':[
            "cd /demo",
            "rm -f /tmp/sagemaker_edge_agent_example.sock",
            "./bin/sagemaker_edge_agent_binary -a /tmp/sagemaker_edge_agent_example.sock -c sagemaker_edge_config.json" 
        ]
    }
)

**아래를 실행하면 `Status`가 `InProgress` 입니다. 이는 현재 agent 데몬형태로 실행되고 있음을 의미 합니다.**

In [44]:
ssm_client.get_command_invocation(
    CommandId=agent_out['Command']['CommandId'],
    InstanceId=instance_id,
)

{'CommandId': '7a48b8a5-b0ec-4de4-b1dc-bce8430cb922',
 'InstanceId': 'i-00ed8289303d74580',
 'Comment': '',
 'DocumentName': 'AWS-RunShellScript',
 'DocumentVersion': '',
 'PluginName': 'aws:runShellScript',
 'ResponseCode': -1,
 'ExecutionEndDateTime': '',
 'Status': 'InProgress',
 'StatusDetails': 'InProgress',
 'StandardOutputContent': '',
 'StandardOutputUrl': '',
 'StandardErrorContent': '',
 'StandardErrorUrl': '',
 'CloudWatchOutputConfig': {'CloudWatchLogGroupName': '',
  'CloudWatchOutputEnabled': False},
 'ResponseMetadata': {'RequestId': '431dfbb9-cded-42cf-b9b0-8a86795e343a',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Mon, 22 Mar 2021 07:42:49 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '471',
   'connection': 'keep-alive',
   'x-amzn-requestid': '431dfbb9-cded-42cf-b9b0-8a86795e343a'},
  'RetryAttempts': 0}}

EC2에 접근하여 실행되고 있는 프로세스를 보면, 아래 그림과 같이 프로세스가 올라와 있습니다.

![MonitorAgent.png](img/MonitorAgent.png)

## 모델 로딩
이 섹션에서는 SageMaker 에지 관리자에서 제공하는 모델 관리 기능을 보여 줍니다.SageMaker 에지 에이전트를 사용하여 컴파일되고 패키지된 모델을 로드합니다.이렇게하면 모델 추론을 실행할 준비가됩니다.보시다시피 모델이 로드되면 모델이 언로드될 때까지 필요한만큼 여러 번 추론을 실행할 수 있습니다.이를 통해 클라이언트 애플리케이션을 별도로 관리해야 하는 로직 및 운영 부담에서 벗어납니다.이 모델은 추론이 가능하다는 단순한 API 예 입니다.

SageMaker Edge 에이전트를 사용하여 모델을 로드할 때 API에 대한 인수는 에이전트를 패키지된 모델을 포함하는 디렉터리 (디렉터리 내에 관계없는 파일이 없게함.) 로 가리킵니다. 

In [45]:
load_keras_model_out = ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName="AWS-RunShellScript",
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Parameters={
        'commands':[
            "cd /demo",
            "./bin/sagemaker_edge_agent_client_example LoadModel keras_model demo-keras"
        ]
    }
)

In [47]:
ssm_client.get_command_invocation(
    CommandId=load_keras_model_out['Command']['CommandId'],
    InstanceId=instance_id,
)

{'CommandId': 'd8a2c43e-7b69-431f-a528-db0475cdbc85',
 'InstanceId': 'i-00ed8289303d74580',
 'Comment': '',
 'DocumentName': 'AWS-RunShellScript',
 'DocumentVersion': '',
 'PluginName': 'aws:runShellScript',
 'ResponseCode': 0,
 'ExecutionStartDateTime': '2021-03-22T07:44:42.665Z',
 'ExecutionElapsedTime': 'PT0.438S',
 'ExecutionEndDateTime': '2021-03-22T07:44:42.665Z',
 'Status': 'Success',
 'StatusDetails': 'Success',
 'StandardOutputContent': 'Model demo-keras located at /demo/keras_model loaded\nLoadModel succeeded\n',
 'StandardOutputUrl': 'https://s3.us-east-2.amazonaws.com/sagemaker-us-east-2-638683464862/DEMO-Sandbox-Sagemaker-Edge/d8a2c43e-7b69-431f-a528-db0475cdbc85/i-00ed8289303d74580/awsrunShellScript/0.awsrunShellScript/stdout',
 'StandardErrorContent': '',
 'StandardErrorUrl': 'https://s3.us-east-2.amazonaws.com/sagemaker-us-east-2-638683464862/DEMO-Sandbox-Sagemaker-Edge/d8a2c43e-7b69-431f-a528-db0475cdbc85/i-00ed8289303d74580/awsrunShellScript/0.awsrunShellScript/stderr

## Predict 실행

이 API에서는 모델 이름, 신경망에 직접 공급되는 입력 데이터 파일, 컴파일 단계에서 이전에 전달 된 입력 텐서 이름, 크기 및 모양을 전달합니다.

#### Run prediction on keras model

In [48]:
keras_predict_out = ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName="AWS-RunShellScript",
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Parameters={
        'commands':[
            "cd /demo",
            "./bin/sagemaker_edge_agent_client_example Predict demo-keras keras.bmp input_1 224 224 3"
        ]
    }
)

In [50]:
ssm_client.get_command_invocation(
    CommandId=keras_predict_out['Command']['CommandId'],
    InstanceId=instance_id,
)

{'CommandId': '2920a409-46ef-4418-b7fc-b7f6b72659ac',
 'InstanceId': 'i-00ed8289303d74580',
 'Comment': '',
 'DocumentName': 'AWS-RunShellScript',
 'DocumentVersion': '',
 'PluginName': 'aws:runShellScript',
 'ResponseCode': 0,
 'ExecutionStartDateTime': '2021-03-22T07:45:01.691Z',
 'ExecutionElapsedTime': 'PT0.18S',
 'ExecutionEndDateTime': '2021-03-22T07:45:01.691Z',
 'Status': 'Success',
 'StatusDetails': 'Success',
 'StandardOutputContent': 'Done reading the image\nPredict succeeded\nFlattened RAW Output Tensor:1\n9.15062e-05 0.000100904 5.23973e-05 7.0895e-05 0.000303288 6.65647e-06 5.73232e-05 0.000241128 7.62334e-05 7.42452e-05 0.000110309 4.28148e-05 7.65651e-05 0.000134373 0.000126301 0.000141554 0.000173317 3.4594e-05 6.91597e-05 3.83557e-05 0.000239026 4.7591e-05 7.32565e-05 0.000167224 8.44282e-05 8.91072e-05 0.000395355 0.000366951 0.000141887 1.91341e-05 0.000868919 0.000156764 5.34357e-05 3.37808e-05 6.51875e-05 0.00017874 0.000960836 0.000114277 0.000148988 0.000183138 

### Model 언로딩

모델을 언로드 한 후 향후 'loadModel` API 호출에 동일한 이름을 다시 사용할 수 있습니다.

In [51]:
# unload_model_out = ssm_client.send_command(
#     InstanceIds=[instance_id],
#     DocumentName="AWS-RunShellScript",
#     OutputS3BucketName=bucket,
#     OutputS3KeyPrefix=folder,
#     Parameters={
#         'commands':[
#             "cd /demo",
#             "./bin/sagemaker_edge_agent_client_example UnloadModel demo-keras"
#         ]
#     }
# )

In [52]:
# ssm_client.get_command_invocation(
#     CommandId=unload_model_out['Command']['CommandId'],
#     InstanceId=instance_id,
# )